<center>
<a href="http://www.udem.edu.co/"><img src="Escudo.png"></a>
<h1>Reconocimiento de Patrones I y II</h1>
<h2>Clasificador discriminante de Fisher</h2>
<h3>2018-2</h3>
</center>

El modelo clasificador discriminante de Fisher es un modelo lineal que permite resolver problemas de clasificación (reconocimiento de patrones) a partir de la búsqueda de una frontera de decisión (hiperplano) que separa las clases del problema (Así como regresión logística, feed-forward neural networks, support vector machines).

<img src="class_problem.jpg">

El clasificador discriminante de Fisher, usa información de variables estadísticas (vectores de media y matrices de varianza y covarianza) para poder trazar el modelo que nos servirá para discriminar nuestras clases. Su regla (condición de decisión) de clasificación es la siguiente:

$W'X < W'*(\frac{\mu_1+\mu_2}{2})$

Donde:

$W = Sw^{-1}*(\mu_1-\mu_2)$

$W’ = $ transpuesta de $W$

$Sw = ((n1 -1) / n1 + n2 -2)*S1 +  ((n2 -1) / n1 + n2 -2  )*S2$

$S1 , S2 = $ Matriz de covarianzas para la clase 1 y la clase 2
 
$n1, n2 =$ cantidad de elementos de la clase 1 y cantidad de elementos de la clase 2

$\mu_1, \mu_2$ = vectores de medias para las clases 1 y 2 respectivamente.

Nota: a $Sw$ se le conoce como matriz de intra-grupos combinada.

### Ejemplos y Ejercicios

Suponga que tiene una base de datos como la que se muestra en la siguiente figura:

<img src="Data.png">

In [1]:
import numpy as np
U0X1 = [2.5, 3, 1, 4, 2]
U1X1 = [10.5, 7, 4.5, 6.5, 7.5]

U0X2 = [12.5, 8.5, 5.4, 15, 8.5]
U1X2 = [2.5, 2, 0.5, 1, 2.5]


U1X1_media = np.mean(U1X1)
U1X2_media = np.mean(U1X2)

valx1 = 0
for num in U1X1:
    valx1 += (num - U1X1_media)**2

valx2 = 0
for num in U1X2:
    valx2 += (num - U1X2_media)**2
    
a = 0
for x1, x2 in zip(U1X1, U1X2):
    a += (x1 - U1X1_media)*(x2 - U1X2_media)


s1 = np.array([[1/5 * valx1 , 1/5 * a], [1/5 * a, 1/5 * valx2]])
s2 = np.array([[1, 2.88], [2.88, 11.38]])
sw = s1*0.5 + 0.5*s2

sw_inversa = np.linalg.inv(sw)

u = np.array([-4.7, 8.3]).reshape(2,1)
a = sw_inversa * u

a.transpose()

array([[-2.846887  , -1.7495969 ],
       [ 0.99073559,  1.98760889]])

In [2]:
(1/5)*(2.25+6.25+2.25+21.16+25)


11.382

### Ejemplo en Python

Trabajaremos con la base de datos IRIS.

In [3]:
import numpy as np
from sklearn import datasets


db = datasets.load_iris()
X = db.data
Y = db.target
#print(db.target_names)

### Inicialmente con un problema biclase

Tomaremos las clases setosa y versicolor

In [4]:
X = X[0:100,:]
Y = Y[0:100]
#print(X.shape)

Separamos las muestras de ambas clases

In [5]:
# Muestras de entrenamientos

Xsetosa = X[0:40,:]
Ysetosa = Y[0:40]

Xversicolor = X[50:90,:]
Yversicolor = Y[50:90]

# Muestras para validación

Xsetosa_test = X[40:50,:]
Ysetosa_test = Y[40:50]

Xversicolor_test = X[90:100,:]
Yversicolor_test = Y[90:100]

Calculamos los elementos que se requieren para aplicar el método clasificador discriminante de Fisher.

In [6]:
#vectores de medias de ambas clases

mu1 = np.mean(Xsetosa, axis=0)
mu2 = np.mean(Xversicolor, axis=0)
print('Vector de medias setosa = ', mu1)
print('Vector de medias versicolor = ', mu2)

Vector de medias setosa =  [5.0375 3.44   1.4625 0.2325]
Vector de medias versicolor =  [6.01   2.78   4.3175 1.35  ]


In [7]:
#Matrices de covarianzas S1 y S2

S1 = np.cov(Xsetosa, rowvar=False)
S2 = np.cov(Xversicolor, rowvar=False)

print('S1 (setosa) = \n', S1)
print('S2 (versicolor) = \n', S2)

S1 (setosa) = 
 [[0.13112179 0.09897436 0.01298077 0.01362179]
 [0.09897436 0.13271795 0.00205128 0.0145641 ]
 [0.01298077 0.00205128 0.02958333 0.00458333]
 [0.01362179 0.0145641  0.00458333 0.00994231]]
S2 (versicolor) = 
 [[0.27374359 0.08661538 0.17212821 0.05230769]
 [0.08661538 0.11087179 0.08087179 0.04538462]
 [0.17212821 0.08087179 0.20353205 0.07371795]
 [0.05230769 0.04538462 0.07371795 0.04307692]]


Ahora se computa $Sw$:

$Sw = ((n1 -1) / n1 + n2 -2) S1 +  ((n2 -1) / n1 + n2 -2  ) S2$

In [8]:
n1 = np.size(Xsetosa,0)
n2 = np.size(Xversicolor,0)


Sw = (((n1-1)/(n1+n2-2))*S1) + (((n2-1)/(n1+n2-22))*S2)
print('Sw = \n', Sw)

Sw = 
 [[0.24962986 0.10772856 0.12223176 0.04198331]
 [0.10772856 0.1409107  0.05540495 0.03779929]
 [0.12223176 0.05540495 0.15164943 0.05186063]
 [0.04198331 0.03779929 0.05186063 0.03393667]]


Ahora computamos W

$W= Sw^{-1}  (VectorMediasClase1  -  VectorMediasClase2)$

In [9]:
Swinv = np.linalg.inv(Sw)
#print(Swinv.shape)
#print(mu1.shape)
#print(mu2.shape)
mu1 = mu1.reshape(4,1)
mu2 = mu2.reshape(4,1)
W = np.dot(Swinv,(mu1-mu2))
W = W.reshape(4,1)
print('W = \n', W)


W = 
 [[  1.27310256]
 [ 18.10241936]
 [-16.27838645]
 [-29.79079159]]


Ahora computamos la salida del modelo

In [14]:
#print(W.T.shape)
#print(((mu1+mu2)/2).shape)
Modelo = np.dot(W.T,((mu1+mu2)/2))
print(Modelo)
print('Salida del modelo = \n', Modelo[0][0])

[[-7.28567624]]
Salida del modelo = 
 -7.285676242345488


Para saber cómo determinar si una nueva muestra es clase 0 (setosa) o clase 1 (versicolor), se prueba una muestra de training en el modelo. 

In [11]:
#Probemos con una muestra de la clase setosa a ver que nos entrega.

prueba_setosa = np.dot(W.T, Xsetosa[10,:])
print(prueba_setosa)

[43.47796744]


In [12]:
#Probemos con una muestra de la clase versicolor a ver que nos entrega.

prueba_versicolor = np.dot(W.T, Xversicolor[10,:])
print(prueba_versicolor)

[-44.19479267]


Ya sabemos entonces que si a la salida obtenemos un valor mayor que -7.2856, entonces la muestra estará por encima de la frontera y será de la clase setosa, en caso contrario estará por debajo de la frontera y será de la clase versicolor.

### Probando el clasificador con las muestras de test

In [13]:
TP,FP,TN,FN = 0,0,0,0
Sensibilidad = 0
Especificidad = 0
Eficiencia = 1


for i in Xsetosa_test:
    salida = np.dot(W.T,i)
    if salida > -7.2856:
        TN += 1
    else:
        FP += 1
        
for j in Xversicolor_test:
    salida = np.dot(W.T,j)
    if salida <= -7.2856:
        TP += 1
    else:
        FN += 1
        
Sensibilidad = TP/(TP+FN)
Especificidad = TN/(TN+FP)
Eficiencia = (TP+TN)/(TP+TN+FP+FN)

print('Resultados:\n\nEficiencia = ', Eficiencia, '\nSensibilidad = ', Sensibilidad, '\nEspecificidad = ', Especificidad)
 

Resultados:

Eficiencia =  1.0 
Sensibilidad =  1.0 
Especificidad =  1.0


## ¿Qué pasa si tengo un problema multiclase?

Existen diferentes estrategias para abordar este problema, con el fin de resolver problemas multiclase con modelos que inicialmente han sido diseñados para resolver problemas biclase. En el curso abordaremos dos estrategias:

1. One vs. One
2. One vs. All

Material desarrollado en clase.

## Ejercicio

Resolver el problema de clasificación de la base de datos IRIS con el modelo clasificador discrimiante de Fisher, aplicando la estrategia One vs. One.